In [ ]:
from Brain import *
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from Env import create_env
from Hype import *

In [ ]:
env = create_env('Breakout')
from torch.utils.tensorboard import SummaryWriter

tb = SummaryWriter('runs/breakout/Second_double_shrinked_buffer')

In [ ]:
save_path = 'Trained_param/Breakout/DoubleDQN'

In [ ]:
# target_net = TargetNetworkBrain(env).cuda()
double_net = DoubleDQNBrain(env).cuda()
# dueling_net = DuelingNetworkBrain(env).cuda()

In [ ]:
# target_optim = optim.RMSprop(target_net.learnable(), lr=LEARNING_RATE, momentum=GRADIENT_MOMENTUM, alpha=SQUARED_GRADIENT_MOMENTUM, eps=MIN_SQUARED_GRADIENT)
double_optim = optim.RMSprop(double_net.learnable(), lr=LEARNING_RATE, momentum=GRADIENT_MOMENTUM, alpha=SQUARED_GRADIENT_MOMENTUM, eps=MIN_SQUARED_GRADIENT)
# dueling_optim = optim.RMSprop(dueling_net.learnable(), lr=LEARNING_RATE, momentum=GRADIENT_MOMENTUM, alpha=SQUARED_GRADIENT_MOMENTUM, eps=MIN_SQUARED_GRADIENT)
criterian = nn.MSELoss()

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple, deque

Transition = namedtuple( 'Transition', ('s1', 'a1', 'r1', 's2', 'done') )

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
replay_memory = ReplayMemory(REPLAY_MEMORY_SIZE)

In [ ]:
s1 = env.reset()
for step in range(REPLAY_START_SIZE):
# for step in range(33):
    a1 = env.action_space.sample()
    s2, r1, done, info = env.step(a1)
    replay_memory.push(s1, a1, r1, s2, done)
    if done:
        s1 = env.reset()
    else:
        s1 = s2
env.close()

In [ ]:
step = 0
for episode in range(1, FINAL_EXPLORATION_FRAME+1):
# for episode in range(1, 300+1):
    print(f'Episode: {episode}')
    total_reward = 0
    total_target_loss = 0
    total_double_loss = 0
    total_dueling_loss = 0
    s1 = env.reset()
    while True:
        step += 1
        # env.render()
        epsilon = np.interp(step, [0, FINAL_EXPLORATION_FRAME], [INITIAL_EXPLORATION, FINAL_EXPLORATION])
        if epsilon > random.random():
            a1 = env.action_space.sample()
        else:
            a1 = double_net.get_action(phi(s1))
        s2, r1, done, info = env.step(a1)
        total_reward += r1
        replay_memory.push(s1, a1, r1, s2, done)
        s1 = s2
        if step % UPDATE_FREQUENCY == 0: # update
            transitions = replay_memory.sample(MINIBATCH_SIZE)
            s1_np = np.asarray([(t.s1) for t in transitions])
            a1_np = np.asarray([(t.a1) for t in transitions])
            r1_np = np.asarray([(t.r1) for t in transitions]).astype('float32')
            s2_np = np.asarray([(t.s2) for t in transitions])
            done_np = np.asarray([(t.done) for t in transitions])
            # y_target = (torch.as_tensor(r1_np)).cuda() + (torch.as_tensor(1 - done_np)).cuda() * DISCOUNT_FACTOR * target_net.get_td(phi(s2_np))
            y_double = (torch.as_tensor(r1_np)).cuda() + (torch.as_tensor(1 - done_np)).cuda() * DISCOUNT_FACTOR * double_net.get_td(phi(s2_np))
            # y_dueling = (torch.as_tensor(r1_np)).cuda() + (torch.as_tensor(1 - done_np)).cuda() * DISCOUNT_FACTOR * dueling_net.get_td(phi(s2_np))
            # q_target = target_net.get_Q(phi(s1_np), phi(a1_np).long())
            q_double = double_net.get_Q(phi(s1_np), phi(a1_np).long())
            # q_dueling = dueling_net.get_Q(phi(s1_np), phi(a1_np).long())
            # loss_target = criterian(y_target, q_target)
            loss_double = criterian(y_double, q_double)
            # loss_dueling = criterian(y_dueling, q_dueling)
            # target_optim.zero_grad()
            double_optim.zero_grad()
            # dueling_optim.zero_grad()
            # loss_target.backward()
            loss_double.backward()
            # loss_dueling.backward()
            # target_optim.step()
            double_optim.step()
            # dueling_optim.step()
            # total_target_loss += loss_target.item()
            total_double_loss += loss_double.item()
            # total_dueling_loss += loss_dueling.item()
        if step % TARGET_NETWORK_UPDATE_FREQUENCY == 0:
            print('Updating Target net', step)
            # target_net.update()
            double_net.update()
        if done:
            tb.add_scalar('Loss', total_double_loss, episode)
            # tb.add_scalar('Double Net Loss', total_double_loss, episode)
            # tb.add_scalar('Dueling Net Loss', total_dueling_loss, episode)
            print(f'Step = {step}')
            print(f'Score = {total_reward}')
            tb.add_scalar('Score', total_reward, episode)
            break
    if episode <= 500:
        if episode % 50 == 0:
            torch.save(double_net.state_dict(), f'{save_path}{episode}_trains.dqn')
    elif episode <= 1000:
        if episode % 100 == 0:
            torch.save(double_net.state_dict(), f'{save_path}{episode}_trains.dqn')
    elif episode <= 5000:
        if episode % 500 == 0:
            torch.save(double_net.state_dict(), f'{save_path}{episode}_trains.dqn')
    elif episode <= 10000:
        if episode % 1000 == 0:
            torch.save(double_net.state_dict(), f'{save_path}{episode}_trains.dqn')
    elif episode <= 50000:
        if episode % 5000 == 0:
            torch.save(double_net.state_dict(), f'{save_path}{episode}_trains.dqn')
    else:
        if episode % 10000 == 0:
            torch.save(double_net.state_dict(), f'{save_path}{episode}_trains.dqn')
env.close()

# 試玩一下

In [ ]:
for i in range(5):
    done = False
    total_reward = 0
    state = env.reset()
    while not done:
        env.render()
        print('steping', end=' ')
        action = dueling_net.get_action(phi(state))
        print(action)
        state, reward, done, info = env.step(action)
        total_reward += reward
    env.close()
    print(total_reward)